In [ ]:
from collections import OrderedDict
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from mshr import *
from DarcySolver import solve_darcy
from BiotSolver import solve_biot
from Optimization import optimize_network_source
from PlottingHelper import (plot_pressures_cross_section, 
                            plot_pressures_and_forces_cross_section,
                            plot_pressures_timeslice,
                            extract_cross_section, style_dict)

# time stepping
T = 1.2           # final time
num_steps = 12    # number of time steps
dt = T/ num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 15*(1e-9)**2       # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-4          # storage coefficent
alpha = 1.0         # Biot-Willis coefficient


# Biot material parameters
E = 1500.0          # Young modulus
nu = 0.479         # Poisson ratio

material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K
material_parameter["lmbda"] = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
material_parameter["mu"] = E/(2.0*(1.0+nu))
material_parameter["alpha"] = alpha
mmHg2Pa = 132.32

# create mesh and mark boundaries
N = 10 # resolution
brain_radius = 0.1 
ventricle_radius = brain_radius/3
brain = Circle(Point(0,0), brain_radius)
ventricle = Circle(Point(0,0), ventricle_radius)
brain = brain - ventricle
mesh = Mesh(generate_mesh(brain, N))

ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R*0.95)",
                              R =brain_radius )
skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.95 )",
                          R = brain_radius)
boundary_marker = MeshFunction("size_t", mesh, mesh.topology().dim()-1, value=0)
skull.mark(boundary_marker, 1)
ventricle.mark(boundary_marker, 2) 
x_coords = np.linspace(ventricle_radius, brain_radius, 20)
slice_points = [Point(x, 0.0) for x in x_coords]

# set analytical expressions
f = 1
A = 1/(brain_radius - ventricle_radius)*mmHg2Pa
p_obs = Expression("A*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*sin(2*pi*f*t)",
                    A=A, f=f, t=0, R_vent=ventricle_radius, degree=2)
p_N =  Expression("A*sin(2*pi*f*t)", A=A, f=f, t=0, R_vent=ventricle_radius, degree=2)

f_ana_darcy = Expression("- K*A/(sqrt(x[0]*x[0] + x[1]*x[1]))*sin(2*pi*f*t)" +
                         "+ c*2*A*pi*f*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*cos(2*pi*f*t)",
                         K=K, A=A, c=c, f=f, t=0, R_vent=ventricle_radius, degree=2)


In [ ]:
# Dirichlet BC
n = FacetNormal(mesh)

boundary_conditions_u = {1:{"Dirichlet":Constant((0.0, 0.0))},
                        2:{"Neumann":-n*p_obs}}

boundary_conditions_p = {1:{"Dirichlet":p_obs},
                         2:{"Dirichlet":p_obs}}

res = optimize_network_source(mesh, material_parameter, times, p_obs,
                              boundary_marker, boundary_conditions_p,
                              boundary_marker, boundary_conditions_u)
opt_ctrls_d, opt_solution_d, initial_solution_d = res
opt_biot_pressure_d = [s.split()[2] for s in opt_solution_d]
init_biot_pressure_d = [s.split()[2] for s in initial_solution_d]

In [ ]:
# Robin BC
beta = 0.5
boundary_conditions_u = {1:{"Dirichlet":Constant((0.0, 0.0))},
                        2:{"Neumann":-n*p_obs}}

boundary_conditions_p = {1:{"Robin": (beta, beta*p_obs + p_N)},
                         2:{"Dirichlet":p_obs}}
res = optimize_network_source(mesh, material_parameter, times, p_obs,
                              boundary_marker, boundary_conditions_p,
                              boundary_marker, boundary_conditions_u)

opt_ctrls_r, opt_solution_r, initial_solution_r = res
opt_biot_pressure_r = [s.split()[2] for s in opt_solution_r]


In [ ]:
pressures = {"p_init" : extract_cross_section(init_biot_pressure_d, slice_points)/mmHg2Pa,
             "p_opt_dirichlet" : extract_cross_section(opt_biot_pressure_d, slice_points)/mmHg2Pa,
             "p_opt_robin" : extract_cross_section(opt_biot_pressure_r, slice_points)/mmHg2Pa,
             "p_obs": extract_cross_section(p_obs, slice_points, times=times)/mmHg2Pa,
             #"p_ana": extract_cross_section(ana_solution, slice_points)/mmHg2Pa
            }
#cdpdt = np.diff(pressures["p_opt_dirichlet"],n=1, axis=0, prepend=0)/dt*c

forces = {"f_opt_dirichlet": extract_cross_section(opt_ctrls_d, slice_points),
          "f_opt_robin": extract_cross_section(opt_ctrls_r, slice_points),
          #"f_ana": extract_cross_section(f_ana, slice_points, times=np.array(times) - 0.5*dt),
          #"c*dp_opt_robin/dt":cdpdt
         }

style_dict["c*dp_opt_robin/dt"] = {"ls":":", "lw":5}

In [ ]:

for i in range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, forces, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f}")

In [ ]:
p_opt